\begin{center}
Gabe Morris
\end{center}

In [1]:
# Notebook Preamble
import matplotlib.pyplot as plt
import sympy as sp
from IPython.display import display

plt.style.use('maroon_ipynb.mplstyle')

\pagebreak
\tableofcontents
\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 4403 & Homework 4 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 6-1
## Given
A 10-mm steel drill rod was heat treated and ground. The measured hardness was found to be 300 Brinell.

## Find
Estimate the endurance strength in $MPa$ if the rod is used in rotating bending.

## Solution
Eq. 6-10 on p. 305, 

$$S^\prime_e=\begin{cases}
0.5S_{ut} & S_{ut} \le 200\ ksi\ (1400\ MPa)\\
100 & S_{ut} > 200\ ksi \\
700\ MPa & S_{ut} > 1400\ MPa
\end{cases}$$

The ultimate strength of steel comes from Eq. 2-36,

$$S_{ut}=3.4H_B$$

In [2]:
H_B = 300
S_ut = sp.S('3.4')*H_B

if S_ut <= 1400:
    S_e_prime = 0.5*S_ut
else:
    S_e_prime = sp.S(700)

S_e_prime  # ksi

510.000000000000

This value is not the final value. The relationship for the refined value is,

$$S_e=k_ak_bk_ck_dk_eS^\prime_e$$

The only necessary $k$ values used for this analysis is $k_a$ and $k_b$, whose equations are at 6-18 and 6-19 respectfully.

In [3]:
# See Table 6-2
k_a = sp.S('1.38')*S_ut**-(sp.S('0.067'))
d = 10
k_b = sp.S('1.24')*d**-(sp.S('0.107'))
# display(k_a, k_b)
S_e = k_a*k_b*S_e_prime
S_e  # MPa

428.839455736079

\pagebreak

\begin{center}
\begin{tabular}{ccc}
ME 4403 & Homework 4 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 6-3
## Given
A steel rotating beam test specimen has an ultimate strength of $120\ ksi$.

## Find
Estimate the life of the specimen if it is tested at completely reversed stress amplitude of $70\ ksi$.

## Solution
Find $S_e$ first.

In [4]:
S_ut = sp.S(120)  # ksi

if S_ut <= 200:
    S_e_prime = 0.5*S_ut
else:
    S_e_prime = sp.S(100)

S_e_prime  # ksi

60.0000000000000

The $S^\prime_e$ value will be used in place of $S_e$ from Figure 6-23 description. We can use the following relationships to determine $N$.

$$N=\left(\frac{\sigma_{ar}}{a}\right)^{1/b}$$
$$a=\frac{(fS_{ut})^2}{S_e}$$
$$b=-\frac{1}{3}\log\left(\frac{fS_{ut}}{Se}\right)$$

The value of $f$ is 0.82 from Figure 6-23. The $S_{ut}$ value is $2(S_e)=120\ ksi$.

In [5]:
def log10(x_):
    return sp.log(x_)/sp.log(10)

f = sp.S('0.82')
a = (f*S_ut)**2/S_e_prime
b = -sp.Rational(1, 3)*log10(f*S_ut/S_e_prime)

display(sp.Eq(sp.Symbol('a'), a.n()),
        sp.Eq(sp.Symbol('b'), b.n()))

sig_ar = 70
N = ((sig_ar/a)**(1/b)).n()
N  # cycles

Eq(a, 161.376)

Eq(b, -0.0716146160158993)

116192.956004683

\pagebreak

\begin{center}
\begin{tabular}{ccc}
ME 4403 & Homework 4 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 6-17
## Given